In [ ]:
! pip3 install langchain langchain-community langchain-text-splitters deepeval python-dotenv pandas 
# chromadb tiktoken boto3 aiobotocore botocore 

In [3]:
#import boto3
from dotenv import load_dotenv, find_dotenv
import os
from deepeval.models import GPTModel
import re
# import random

In [4]:
load_dotenv(find_dotenv())

True

In [5]:
# # === AWS Configuration === #
# COGNITO_REGION = os.getenv("COGNITO_REGION")
# BEDROCK_REGION = os.getenv("BEDROCK_REGION")
# MODEL_ID1 = os.getenv("MODEL_ID1")
# MODEL_ID2 = os.getenv("MODEL_ID2")
# IDENTITY_POOL_ID = os.getenv("IDENTITY_POOL_ID")
# USER_POOL_ID = os.getenv("USER_POOL_ID")
# APP_CLIENT_ID = os.getenv("APP_CLIENT_ID")
# USERNAME = os.getenv("USERNAME")
# PASSWORD = os.getenv("PASSWORD")

In [6]:
# # === Helper: Get AWS Credentials === #
# def get_credentials(username, password):
#     idp_client = boto3.client("cognito-idp", region_name=COGNITO_REGION)
#     response = idp_client.initiate_auth(
#         AuthFlow="USER_PASSWORD_AUTH",
#         AuthParameters={"USERNAME": username, "PASSWORD": password},
#         ClientId=APP_CLIENT_ID,
#     )
#     id_token = response["AuthenticationResult"]["IdToken"]

#     identity_client = boto3.client("cognito-identity", region_name=COGNITO_REGION)
#     identity_response = identity_client.get_id(
#         IdentityPoolId=IDENTITY_POOL_ID,
#         Logins={f"cognito-idp.{COGNITO_REGION}.amazonaws.com/{USER_POOL_ID}": id_token},
#     )

#     creds_response = identity_client.get_credentials_for_identity(
#         IdentityId=identity_response["IdentityId"],
#         Logins={f"cognito-idp.{COGNITO_REGION}.amazonaws.com/{USER_POOL_ID}": id_token},
#     )

#     return creds_response["Credentials"]

In [7]:
# credentials = get_credentials(USERNAME, PASSWORD)

In [8]:
# os.environ["AWS_ACCESS_KEY_ID"]=credentials.get('AccessKeyId')
# os.environ["AWS_SECRET_ACCESS_KEY"]=credentials.get('SecretKey')
# os.environ["AWS_SESSION_TOKEN"]=credentials.get('SessionToken')

In [9]:
# llm = AmazonBedrockModel(
#     model_id=MODEL_ID1,
#     region_name=BEDROCK_REGION,
# )

llm = GPTModel(model="gpt-5-nano", temperature=1.0)

In [10]:
llm.model

In [11]:
llm.model_name

'gpt-5-nano'

In [12]:
# llm.generation_kwargs = {
#     "maxTokens": 200,
#     "temperature":0
# }

In [13]:
# from deepeval.models import OllamaEmbeddingModel

In [14]:
# emb_model = OllamaEmbeddingModel(model="bge-m3:latest")

In [15]:
# emb_model.model_name = "bge-m3:latest"

In [16]:
# emb_model.model_name

In [17]:
#from deepeval.synthesizer.config import ContextConstructionConfig
from deepeval.synthesizer.config import FiltrationConfig
from deepeval.synthesizer.config import EvolutionConfig
from deepeval.synthesizer import Evolution
from deepeval.synthesizer.config import StylingConfig
from deepeval.synthesizer import Synthesizer

In [18]:
filtration_config = FiltrationConfig(
  synthetic_input_quality_threshold=0.8
)

In [19]:
evolution_config = EvolutionConfig(
    evolutions={
        #Evolution.MULTICONTEXT: 1/2, # sticks to the context
        Evolution.CONCRETIZING: 1, # sticks to the context
        #Evolution.CONSTRAINED: 1/4, # sticks to the context
        #Evolution.COMPARATIVE: 1/4, # sticks to the context
    },
    num_evolutions=1
)

In [20]:
styling_config = StylingConfig(
  input_format="Questions in English regarding Canvas LMS navigation, usage and troubleshooting.",
  expected_output_format="Step-by-step instructions to help users navigate, use and troubleshoot Canvas LMS.",
  task="Answering user queries about Canvas LMS and returning the results to users",
  scenario="Non-technical users, new to Canvas LMS, seeking help with navigation, usage and troubleshooting using plain English.",
)

In [21]:
synthesizer = Synthesizer(
    model=llm, 
    async_mode=False, 
    filtration_config=filtration_config, 
    evolution_config=evolution_config, 
    styling_config=styling_config
)

In [22]:
canvas_student_guide = [
    # Introduction
    'https://community.canvaslms.com/t5/Student-Guide/Student-Getting-Started-Resources/ta-p/579371',
    'https://community.canvaslms.com/t5/Student-Guide/How-do-I-accept-an-email-invitation-to-join-a-Canvas-course/ta-p/1563',
    'https://community.canvaslms.com/t5/Student-Guide/How-do-I-sign-up-for-a-Canvas-account-with-a-join-code-or-secret/ta-p/437',
    'https://community.canvaslms.com/t5/Student-Guide/How-do-I-generate-a-pairing-code-for-an-observer-as-a-student/ta-p/418',
    # Announcements
    'https://community.canvaslms.com/t5/Student-Guide/How-do-I-view-Announcements-as-a-student/ta-p/417',
    'https://community.canvaslms.com/t5/Student-Guide/How-do-I-add-an-announcement-in-a-group/ta-p/458',
    'https://community.canvaslms.com/t5/Student-Guide/How-do-I-edit-an-announcement-in-a-group/ta-p/459',
    'https://community.canvaslms.com/t5/Student-Guide/How-do-I-delete-an-announcement-in-a-group/ta-p/457',
    'https://community.canvaslms.com/t5/Student-Guide/How-do-I-add-an-external-RSS-feed-to-a-group-announcement-as-a/ta-p/449',
    'https://community.canvaslms.com/t5/Student-Guide/How-do-I-subscribe-to-the-Announcements-RSS-feed-as-a-student/ta-p/448',
    'https://community.canvaslms.com/t5/Student-Guide/How-do-I-reply-to-an-announcement-as-a-student/ta-p/447',
    'https://community.canvaslms.com/t5/Student-Guide/How-do-I-like-a-reply-in-a-course-announcement-as-a-student/ta-p/344',
    'https://community.canvaslms.com/t5/Student-Guide/How-do-I-view-or-reply-to-an-announcement-thread-as-a-student/ta-p/485667',
    'https://community.canvaslms.com/t5/Student-Guide/How-do-I-mention-a-user-in-an-announcement-reply-as-a-student/ta-p/485670',
    'https://community.canvaslms.com/t5/Student-Guide/How-do-I-view-announcements-as-a-student-in-Canvas-for/ta-p/645000',
    # Assignments
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-view-Assignments-as-a-student/ta-p/451",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-view-the-rubric-for-my-assignment/ta-p/275",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-view-the-rubric-for-my-external-tool-assignment/ta-p/526",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-submit-an-online-assignment/ta-p/503",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-submit-a-text-entry-assignment/ta-p/303",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-enter-a-URL-as-an-assignment-submission/ta-p/286",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-submit-a-media-file-as-an-assignment-submission/ta-p/279",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-submit-a-Lucid-document-for-a-Lucid-assignment/ta-p/606359",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-submit-an-assignment-on-behalf-of-a-group/ta-p/294",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-know-if-I-have-a-peer-review-assignment-to-complete/ta-p/318",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-submit-a-peer-review-to-an-assignment/ta-p/293",
    "https://community.canvaslms.com/t5/Student-Guide/Where-can-I-find-my-peers-feedback-for-peer-reviewed-assignments/ta-p/320",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-upload-a-file-as-an-assignment-submission-in-Canvas/ta-p/274",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-use-my-webcam-to-take-a-photo-for-an-assignment/ta-p/452365",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-annotate-a-file-as-an-assignment-submission-in-Canvas/ta-p/463702",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-upload-a-file-from-Google-Drive-as-an-assignment/ta-p/499",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-submit-a-Google-Assignments-LTI-1-3-file-as-an/ta-p/587649",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-submit-a-cloud-assignment-with-Google-Drive/ta-p/491",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-upload-a-file-from-Microsoft-Office-365-as-an/ta-p/301",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-know-when-my-assignment-has-been-submitted/ta-p/277",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-know-when-my-instructor-has-graded-my-assignment/ta-p/276",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-submit-a-cloud-assignment-with-Microsoft-Office-365/ta-p/299",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-view-assignment-comments-from-my-instructor/ta-p/283",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-use-DocViewer-in-Canvas-assignments-as-a-student/ta-p/525",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-view-annotation-feedback-comments-from-my-instructor/ta-p/523",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-view-rubric-results-for-my-assignment/ta-p/533",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-view-my-Roll-Call-Attendance-report-as-a-student/ta-p/354",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-download-assignment-submissions-from-all-my-courses/ta-p/298",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-submit-an-online-assignment-in-Canvas-for-Elementary/ta-p/644996",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-view-assignment-feedback-from-my-instructor-in-Canvas/ta-p/645001",
    # Assignment Enhancements
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-view-an-assignment-in-a-course-using-Assignment/ta-p/416676",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-view-the-rubric-for-an-assignment-in-a-course-using/ta-p/416677",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-submit-an-online-assignment-in-a-course-using/ta-p/416675",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-submit-an-assignment-on-behalf-of-a-group-using/ta-p/481742",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-submit-a-text-entry-assignment-in-a-course-using/ta-p/416667",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-upload-a-file-as-an-assignment-submission-for-a-course/ta-p/416664",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-annotate-a-file-as-an-assignment-submission-using/ta-p/478067",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-use-my-webcam-to-take-a-photo-for-an-assignment/ta-p/478066",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-submit-a-media-file-as-an-assignment-submission-for-a/ta-p/416670",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-enter-a-URL-as-an-assignment-submission-for-a-course/ta-p/416672",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-submit-a-Google-Drive-file-or-URL-as-an-assignment/ta-p/416665",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-submit-a-Microsoft-Office-365-file-as-an-assignment/ta-p/416674",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-know-when-my-assignment-has-been-submitted-in-a-course/ta-p/416678",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-submit-a-self-assessment-to-an-assignment-using/ta-p/637111",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-know-if-I-have-a-peer-review-assignment-to-complete/ta-p/566790",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-submit-a-peer-review-to-an-assignment-using-Assignment/ta-p/566791",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-submit-a-peer-review-to-an-assignment-using-Enhanced/ta-p/609350",
    "https://community.canvaslms.com/t5/Student-Guide/Where-can-I-find-my-peers-feedback-for-peer-reviewed-assignments/ta-p/566792",
    "https://community.canvaslms.com/t5/Student-Guide/Where-can-I-find-my-peers-feedback-for-peer-reviewed-assignments/ta-p/609351",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-view-assignment-feedback-from-my-instructor-using/ta-p/474046",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-use-DocViewer-in-Canvas-using-Assignment-Enhancements/ta-p/611848",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-submit-an-online-assignment-in-Canvas-for-Elementary/ta-p/644994",
    # Chat
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-use-Chat-as-a-student/ta-p/315",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-use-inline-chat-as-a-student/ta-p/442",
    # Collaborations
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-view-Collaborations-as-a-student/ta-p/413",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-create-a-Google-Docs-collaboration-as-a-student/ta-p/391",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-create-a-Google-Drive-collaboration-as-a-student/ta-p/389",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-create-a-Google-Assignments-LTI-1-3-collaboration-as-a/ta-p/587648",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-create-a-Microsoft-Office-365-collaboration-as-a/ta-p/390",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-delete-a-collaboration-as-a-student/ta-p/387",
    # Conferences
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-use-Conferences-in-a-course-as-a-student/ta-p/470",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-create-a-conference-in-a-student-group/ta-p/270",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-edit-or-delete-a-conference-in-a-student-group/ta-p/343",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-record-a-conference-in-a-student-group/ta-p/271",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-start-a-conference-in-a-student-group/ta-p/348",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-join-a-conference-in-a-course-as-a-student/ta-p/332",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-use-the-Conferences-interface-as-a-participant/ta-p/342",
    # Course Navigation
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-use-the-Course-Home-Page-as-a-student/ta-p/504",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-use-the-Course-Activity-Stream-as-a-student/ta-p/394",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-drop-a-self-enrolled-Canvas-course/ta-p/454",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-use-the-Syllabus-as-a-student/ta-p/439",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-view-Course-Analytics-in-a-course-as-a-student/ta-p/450",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-use-the-Microsoft-Immersive-Reader-in-a-course-as-a/ta-p/446",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-use-my-Homeroom-as-a-student-in-Canvas-for-Elementary/ta-p/644992",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-view-Important-Dates-as-a-student-in-Canvas-for/ta-p/644995",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-navigate-my-subject-as-a-student-in-Canvas-for/ta-p/644993",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-drop-a-self-enrolled-Canvas-for-Elementary-subject/ta-p/644997",
    # Discussions
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-view-Discussions-as-a-student/ta-p/314",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-view-the-rubric-for-my-graded-discussion/ta-p/319",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-subscribe-to-a-discussion-podcast-as-a-student/ta-p/368",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-know-if-I-have-a-peer-review-discussion-to-complete/ta-p/419",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-submit-a-peer-review-to-a-discussion/ta-p/355",
    "https://community.canvaslms.com/t5/Student-Guide/Where-can-I-find-my-peers-feedback-for-peer-reviewed-discussions/ta-p/428",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-create-a-course-discussion-as-a-student/ta-p/300",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-subscribe-to-a-discussion-as-a-student/ta-p/352",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-view-and-sort-discussion-replies-as-a-student/ta-p/465",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-change-discussion-settings-to-manually-mark-discussion/ta-p/366",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-mark-discussion-replies-as-read-or-unread-as-a-student/ta-p/284",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-reply-to-a-discussion-as-a-student/ta-p/334",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-attach-a-file-to-a-discussion-reply-as-a-student/ta-p/375",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-embed-an-image-in-a-discussion-reply-as-a-student/ta-p/313",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-edit-or-delete-discussion-replies-as-a-student/ta-p/399",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-like-a-reply-in-a-course-discussion-as-a-student/ta-p/392",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-view-a-discussion-thread-as-a-student/ta-p/485668",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-mention-a-user-in-a-discussion-reply-as-a-student/ta-p/485669",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-report-a-reply-in-a-discussion/ta-p/542169",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-reply-to-a-discussion-as-a-student-in-Canvas-for/ta-p/645002",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-translate-a-discussion-using-AI-Translations-as-a/ta-p/660442",
    # Grades
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-view-my-grades-in-a-current-course/ta-p/493",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-add-a-comment-to-a-submission-as-a-student/ta-p/606650",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-view-my-Learning-Mastery-scores-in-the-Grades-page/ta-p/477",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-use-the-icons-and-colors-in-the-Grades-page/ta-p/476",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-approximate-my-assignment-scores-using-the-What-If/ta-p/481",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-view-my-grades-in-a-concluded-course/ta-p/492",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-view-my-grades-in-a-current-subject-in-Canvas-for/ta-p/644998",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-view-my-Learning-Mastery-scores-for-a-subject-in-Canvas/ta-p/644999",
    # Modules
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-view-Modules-as-a-student/ta-p/433",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-mark-a-module-item-as-done-for-a-module-requirement/ta-p/311",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-choose-an-assignment-path-in-a-module/ta-p/478",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-view-course-content-offline-as-an-HTML-file-as-a/ta-p/341",
    # Pages
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-view-Pages-as-a-student/ta-p/371",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-create-a-new-page-in-a-group/ta-p/304",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-change-a-page-name-in-a-group/ta-p/362",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-edit-a-course-page-as-a-student/ta-p/479",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-edit-a-page-in-a-group/ta-p/487",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-delete-pages-in-a-group/ta-p/330",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-upload-a-PDF-to-a-page-in-a-group/ta-p/347",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-link-to-other-pages-in-a-group/ta-p/349",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-embed-media-in-a-page-in-a-group/ta-p/508",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-create-a-file-link-in-a-page-in-a-group/ta-p/510",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-view-the-history-of-a-page-in-a-group/ta-p/404",
    # People and Groups
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-use-the-People-page-in-a-course-as-a-student/ta-p/530",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-view-my-course-analytics-as-a-student/ta-p/312",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-view-all-groups-in-a-course-as-a-student/ta-p/353",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-view-my-Canvas-groups-as-a-student/ta-p/281",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-use-groups-as-a-student/ta-p/603040",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-join-a-group-as-a-student/ta-p/468",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-leave-a-group-as-a-student/ta-p/415",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-switch-groups-as-a-student/ta-p/601381",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-create-a-group-as-a-student/ta-p/280",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-upload-a-file-to-a-group/ta-p/278",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-start-a-collaboration-in-a-group-as-a-student/ta-p/438",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-create-a-discussion-in-a-group-as-a-student/ta-p/350",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-close-a-discussion-for-comments-in-a-group-as-a-student/ta-p/376",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-manage-groups-as-a-student-group-leader/ta-p/473",
    # Portfolios
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-use-Portfolios-as-a-student/ta-p/651576",
    # Quizzes
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-view-Quizzes-as-a-student/ta-p/472",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-view-the-rubric-for-a-quiz/ta-p/453",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-take-a-quiz/ta-p/507",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-take-a-quiz-in-New-Quizzes/ta-p/291",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-take-a-quiz-where-I-can-only-view-one-question-at-a/ta-p/482",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-take-a-quiz-where-I-can-only-view-one-question-at-a/ta-p/292",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-answer-each-type-of-question-in-a-quiz/ta-p/474",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-answer-each-type-of-question-in-New-Quizzes/ta-p/290",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-resume-a-quiz-that-I-already-started-taking/ta-p/452",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-submit-a-quiz/ta-p/475",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-view-quiz-results-as-a-student/ta-p/335",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-view-quiz-comments-from-my-instructor/ta-p/471",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-view-my-quiz-results-as-a-student-in-New-Quizzes/ta-p/289",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-know-if-I-can-retake-a-quiz/ta-p/490",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-know-if-I-can-retake-a-quiz-in-New-Quizzes/ta-p/287",
    "https://community.canvaslms.com/t5/Student-Guide/How-do-I-submit-a-survey/ta-p/380",
]

In [ ]:
! pip3 install beautifulsoup4

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters.character import RecursiveCharacterTextSplitter
import bs4

In [25]:
class ContextGenerator:
     
    def __init__(self, sources: list[str], chunk_size=4200, chunk_overlap=200):
        self._sources = sources
        self._chunk_size = chunk_size
        self._chunk_overlap = chunk_overlap
        self._text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap, length_function=len)
        self._total_chunks = 0
    
    def _clean_page_content(self, document):
        """Clean video transcript and reference labels"""

        matches = re.findall('[0-9]{2}:[0-9]{2}: [0-9A-Za-z ;.,!?-]*',document.page_content)

        if not matches:
            return document

        page_content = document.page_content
        idx1 = page_content.find(matches[0])
        idx2 = page_content.find(matches[-1]) + len(matches[-1])
        page_content = page_content[:idx1] + page_content[idx2:]
        page_content = re.sub(r'\[\d+\]','',page_content)
        page_content = re.sub('\n',' ',page_content)
        document.page_content = page_content
        
        return document
    
    def _populate_metadata(self, document):

        """Add doc_id and doc_title to metadata"""

        source = document.metadata.get('source')
        document.metadata['doc_id'] = self._sources.index(source)
        document.metadata['doc_title'] = " ".join(source.split("/")[-3].split("-"))

        return document
    
    def _prepend_doc_title(self, document):
        
        """Prepend chunk page_content with parent doc_tile"""

        doc_title = document.metadata.get('doc_title')
        document.page_content = f"Document Title: {doc_title}\n\n" + document.page_content

        return document
    
    def load_contexts(self):

        """Load, clean, split documents from web sources"""

        loader = WebBaseLoader(
            web_paths= self._sources,
            bs_kwargs={
                "parse_only": bs4.SoupStrainer(id="content"),
            },
            bs_get_text_kwargs={"separator": "\n", "strip": True},
        )

        contexts = []

        for doc in loader.lazy_load():
            cleaned_doc = self._clean_page_content(doc)
            cleaned_doc_with_metadata = self._populate_metadata(cleaned_doc)
            chunks = list(map(self._prepend_doc_title, self._text_splitter.split_documents([cleaned_doc_with_metadata])))
            chunks = list(map(lambda doc: doc.page_content.strip(), chunks))
            self._total_chunks += len(chunks)
            contexts.append(chunks)
        
        return contexts

In [26]:
contexts = ContextGenerator(canvas_student_guide).load_contexts()

In [27]:
len(contexts)

167

In [28]:
# greater_than_2 = 0
# greater_than_3 = 0
# for c in contexts:
#     if len(c) > 2:
#         greater_than_2 += 1
#     if len(c) > 3:
#         greater_than_3 += 1

In [29]:
# print(f"Documents with more than 2 chunks: {greater_than_2}")
# print(f"Documents with more than 3 chunks: {greater_than_3}")

In [ ]:
! pip3 install ipywidgets

In [31]:
synthesizer.generate_goldens_from_contexts(
    contexts=contexts,
    include_expected_output=True,
    max_goldens_per_context=1,
) # more params to explore


Output()

[Golden(input='How do I open the Canvas Dashboard and view my assignments as a student?', actual_output=None, expected_output='- Step 1: Open Canvas using your institution’s Canvas URL (or launch the Canvas Student app on your device).\n\n- Step 2: Sign in as a student (use your Canvas credentials). If you’re joining via invitation, accept it; if you have a join code or secret URL, use that to enroll.\n\n- Step 3: Navigate to the Dashboard: After logging in, use the Dashboard to view your courses as tiles.\n\n- Step 4: Open a course to view assignments: Click the course tile to enter the course, then open the Assignments area to see due dates and submission requirements. (Alternatively, you can view assignments from within the course page.)\n\n- Step 5: Troubleshooting if nothing is visible:\n  - Ensure the course is published and has started (students cannot view a course until it’s published and started).\n  - Make sure you are enrolled as a student in the course.\n  - Refresh the pa

In [ ]:
synthesizer.save_as(
	file_type="csv",
	file_name="synthetic_dataset-student-guide",
	directory="../data/datasets"
)

Synthetic goldens saved at ./data/synthetic_dataset.csv!

'./data/synthetic_dataset.csv'

In [33]:
# dataframe = synthesizer.to_pandas()

In [34]:
# dataframeø